In [1]:
import os

DATASET_PATH = r"C:\Users\Dell\.cache\kagglehub\datasets\aryashah2k\brain-tumor-segmentation-brats-2019\versions\1\MICCAI_BraTS_2019_Data_Training\LGG"


print("🔍 Checking Dataset Path...")
print(os.listdir(DATASET_PATH)[:5])

🔍 Checking Dataset Path...
['BraTS19_2013_0_1', 'BraTS19_2013_15_1', 'BraTS19_2013_16_1', 'BraTS19_2013_1_1', 'BraTS19_2013_24_1']


In [2]:
from torch.utils.data import Dataset
import os, glob
import torch
import numpy as np
import nibabel as nib
import torch.nn.functional as F
from torchvision import transforms

class BraTSDataset(Dataset):
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path
        self.modalities = ["flair", "t1", "t1ce", "t2"]
        self.patients = []

        # Pretrained model normalization (ImageNet)
        self.img_transform = transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )

        all_patients = sorted(os.listdir(dataset_path))
        for patient in all_patients:
            patient_path = os.path.join(dataset_path, patient)
            try:
                _ = [self.load_nii(patient_path, mod) for mod in self.modalities]
                _ = self.load_nii(patient_path, "seg")
                self.patients.append(patient)
            except:
                print(f"⚠️ Skipping {patient} in init due to missing files")

    def __len__(self):
        return len(self.patients)

    def load_nii(self, patient_folder, modality):
        possible_files = glob.glob(os.path.join(patient_folder, f"*{modality}*.nii*"))
        if not possible_files and modality == "t1":
            possible_files = glob.glob(os.path.join(patient_folder, f"*ROI*{modality}*.nii*"))
        if not possible_files:
            raise FileNotFoundError(f"❌ No file found for modality '{modality}' in {patient_folder}")
        possible_files.sort(key=lambda x: x.endswith(".gz"), reverse=True)
        file_path = possible_files[0]
        return nib.load(file_path).get_fdata()

    def __getitem__(self, idx):
        patient = self.patients[idx]
        patient_path = os.path.join(self.dataset_path, patient)

        try:
            # Only load the first 3 modalities (flair, t1, t2)
            images = [self.load_nii(patient_path, mod) for mod in self.modalities[:3]]  # [3, 240, 240, 155]
            images = np.stack(images, axis=0)  # [3, 240, 240, 155]

            mask = self.load_nii(patient_path, "seg")  # [240, 240, 155]
            slice_idx = mask.shape[-1] // 2

            # Select the slice and create a 3-channel image
            image = images[:, :, :, slice_idx]  # [3, 240, 240]
            mask = mask[:, :, slice_idx]         # [240, 240]

            # Normalize image to [0, 1]
            image = image.astype(np.float32)
            image = (image - image.min()) / (image.max() - image.min() + 1e-8)

            # Convert to torch tensor
            image_tensor = torch.tensor(image, dtype=torch.float32)  # [3, 240, 240]
            mask_tensor = torch.tensor(mask, dtype=torch.long)       # [240, 240]

            # Resize both to 224x224
            image_tensor = F.interpolate(image_tensor.unsqueeze(0), size=(224, 224), mode="bilinear", align_corners=False).squeeze(0)
            mask_tensor = mask_tensor.unsqueeze(0).unsqueeze(0).float()
            mask_tensor = F.interpolate(mask_tensor, size=(224, 224), mode="nearest").squeeze(0).squeeze(0).long()

            # Apply normalization for pretrained model
            image_tensor = self.img_transform(image_tensor)

            print(f"✅ Loaded {patient}: Image {image_tensor.shape}, Mask {mask_tensor.shape}")
            return image_tensor, mask_tensor

        except FileNotFoundError as e:
            print(f"⚠️ Skipping {patient} due to missing files: {e}")
            return None


# ➕ Custom collate_fn to skip Nones in batches
def skip_none_collate(batch):
    batch = [b for b in batch if b is not None]
    if len(batch) == 0:
        return torch.empty(0), torch.empty(0, dtype=torch.long)
    return torch.utils.data.dataloader.default_collate(batch)


In [3]:
from torch.utils.data import Dataset, DataLoader

In [4]:
import glob
import torch.nn.functional as F

# Load Dataset
train_dataset = BraTSDataset(DATASET_PATH)

valid_samples = [d for d in train_dataset if d is not None]

if len(valid_samples) == 0:
    raise ValueError("❌ No valid samples found. Check dataset paths and file integrity.")


#train_loader = DataLoader(valid_samples, batch_size=2, shuffle=True, num_workers=0)
train_loader = DataLoader(valid_samples, batch_size=4, shuffle=True, collate_fn=skip_none_collate, num_workers=0)
print(f"✅ Loaded {len(train_loader.dataset)} valid patients from BraTS 2019(HGG).")

✅ Loaded BraTS19_2013_0_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_2013_15_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_2013_16_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_2013_1_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_2013_24_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_2013_28_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_2013_29_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_2013_6_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_2013_8_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_2013_9_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_TCIA09_141_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_TCIA09

In [5]:
device = torch.device('cpu')
import torch.nn as nn

In [6]:
def convert_bn_to_gn(model, default_num_groups=32):
    for name, module in model.named_children():
        if isinstance(module, nn.BatchNorm2d):
            num_channels = module.num_features
            # Find the largest divisor of num_channels not exceeding default_num_groups
            num_groups = min(default_num_groups, num_channels)
            while num_channels % num_groups != 0:
                num_groups -= 1
            gn = nn.GroupNorm(num_groups=num_groups, num_channels=num_channels)
            setattr(model, name, gn)
        else:
            convert_bn_to_gn(module, default_num_groups)


import segmentation_models_pytorch as smp

# Create DeepLabV3+ model with ResNet50 encoder
model = smp.DeepLabV3Plus(
    encoder_name="resnet50",        # Choose encoder
    encoder_weights="imagenet",     # Use ImageNet pre-trained weights
    in_channels=3,                  
    classes=4                       
)
convert_bn_to_gn(model)

c:\Users\Dell\Desktop\Group_4_sem_6_AIML\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
model.load_state_dict(torch.load(r"C:\Users\Dell\Desktop\Group_4_sem_6_AIML\comparativeAnalysisModel\DeepLabV3 + ResNet50\saved_models\2019_hgg_deeplabv3_resNet50.pth"))
print("✅ Loaded pretrained weights from 2019(HGG) dataset")

✅ Loaded pretrained weights from 2019(HGG) dataset


In [8]:
from torch.utils.data import random_split, DataLoader

# Split Dataset into 80% Train, 20% Validation
dataset_size = len(train_dataset)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])


train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=0)

print(f"✅ Train Set: {len(train_dataset)} samples | Val Set: {len(val_dataset)} samples")


✅ Train Set: 60 samples | Val Set: 16 samples


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Soft Dice Loss
class SoftDiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(SoftDiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, pred, target):
        pred = torch.softmax(pred, dim=1)  # Apply softmax to logits
        target_one_hot = F.one_hot(target, num_classes=pred.shape[1])  # [B, H, W, C]
        target_one_hot = target_one_hot.permute(0, 3, 1, 2).float()     # [B, C, H, W]

        dims = (0, 2, 3)  # Batch, Height, Width
        intersection = (pred * target_one_hot).sum(dim=dims)
        union = pred.sum(dim=dims) + target_one_hot.sum(dim=dims)
        dice = (2. * intersection + self.smooth) / (union + self.smooth)
        return 1 - dice.mean()

# Combined Dice + CrossEntropy loss for SMP models
def combined_loss(preds, target):
    # If the model only outputs one prediction, adjust the loss function accordingly
    dice_loss = SoftDiceLoss()(preds, target)
    ce_loss = nn.CrossEntropyLoss(weight=torch.tensor([0.1, 1.0, 1.0, 3.0]).to(preds.device))(preds, target)

    total_loss = dice_loss + ce_loss
    return total_loss


In [10]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
import copy

def train_model_deeplab(model, train_loader, optimizer, loss_fn, device, epochs=10, patience=3, save_path="best_deeplabv3.pth"):
    best_loss = float('inf')
    best_model_wts = copy.deepcopy(model.state_dict())
    no_improve_epochs = 0

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{epochs}]", leave=False)

        for images, masks in loop:
            images, masks = images.to(device), masks.to(device)
            masks = torch.clamp(masks, 0, 3)  # Ensure valid class range

            optimizer.zero_grad()

            preds = model(images)  # SMP-style output [B, C, H, W]
            loss = loss_fn(preds, masks)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            loop.set_postfix({"Loss": loss.item()})

        avg_train_loss = train_loss / len(train_loader)
        print(f"\n✅ Epoch {epoch+1}: Train Loss: {avg_train_loss:.4f}")

        if avg_train_loss < best_loss:
            best_loss = avg_train_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, save_path)
            print(f"💾 Saved model at epoch {epoch+1} with train loss {best_loss:.4f}")
            no_improve_epochs = 0
        else:
            no_improve_epochs += 1
            print(f"⚠️ No improvement for {no_improve_epochs} epoch(s)")
            if no_improve_epochs >= patience:
                print(f"⏹️ Early stopping triggered at epoch {epoch+1}")
                break

    model.load_state_dict(best_model_wts)
    print("🏁 Training complete.")


In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
train_model_deeplab(
    model=model,
    train_loader=train_loader,
    optimizer=optimizer,
    loss_fn=combined_loss, 
    device=device,
    epochs=15,
    patience=3,
    save_path="2019_lgg_deeplabv3_resNet50.pth"
)

Epoch [1/15]:   0%|          | 0/30 [00:00<?, ?it/s]

✅ Loaded BraTS19_TCIA10_408_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_TCIA09_451_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])


Epoch [1/15]:   3%|▎         | 1/30 [00:00<00:21,  1.37it/s, Loss=0.376]

✅ Loaded BraTS19_TCIA10_442_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_TCIA10_130_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])


Epoch [1/15]:   7%|▋         | 2/30 [00:01<00:19,  1.47it/s, Loss=0.555]

...

Epoch [11/15]:  93%|█████████▎| 28/30 [00:14<00:01,  1.94it/s, Loss=0.206]

✅ Loaded BraTS19_TCIA10_241_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_TCIA10_640_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])


Epoch [11/15]:  97%|█████████▋| 29/30 [00:15<00:00,  1.96it/s, Loss=0.256]

✅ Loaded BraTS19_TCIA13_621_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_TCIA13_618_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])



✅ Epoch 11: Train Loss: 0.3947
⚠️ No improvement for 3 epoch(s)
⏹️ Early stopping triggered at epoch 11
🏁 Training complete.


In [12]:
import torch
import torch.nn.functional as F

def dice_score(preds, targets, num_classes=4, smooth=1e-6):
    dice_per_class = []
    for cls in range(num_classes):
        pred_cls = (preds == cls).float()
        target_cls = (targets == cls).float()

        intersection = (pred_cls * target_cls).sum()
        union = pred_cls.sum() + target_cls.sum()

        dice = (2 * intersection + smooth) / (union + smooth)
        dice_per_class.append(dice.item())
    return dice_per_class


def evaluate_model(model, val_loader, device, num_classes=4):
    model.eval()
    total_dice = torch.zeros(num_classes).to(device)
    total_batches = 0

    with torch.no_grad():
        for images, masks in val_loader:
            images, masks = images.to(device), masks.to(device)

            # For SMP models, this returns the raw logits [B, C, H, W]
            main_pred = model(images)

            # Resize logits to match mask size if needed
            main_pred = F.interpolate(main_pred, size=masks.shape[1:], mode="bilinear", align_corners=False)

            # Get predicted class per pixel
            preds = torch.argmax(main_pred, dim=1)

            batch_dice = dice_score(preds, masks, num_classes=num_classes)
            total_dice += torch.tensor(batch_dice, device=device)
            total_batches += 1

    avg_dice = total_dice / total_batches

    for cls, score in enumerate(avg_dice):
        print(f"🔹 Dice Score for Class {cls}: {score:.4f}")
    print(f"\n🎯 Mean Dice Score: {avg_dice.mean():.4f}")


In [13]:
evaluate_model(model, val_loader, device)

✅ Loaded BraTS19_2013_29_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_TCIA10_266_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_TCIA10_625_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_TCIA13_645_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_2013_16_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_TCIA09_493_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_2013_28_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_TCIA10_644_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_TCIA09_428_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_2013_0_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅ Loaded BraTS19_TCIA13_650_1: Image torch.Size([3, 224, 224]), Mask torch.Size([224, 224])
✅